In [1]:
# import 
import pandas as pd
import warnings  
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import sys 
import pickle 
import plotly.express as px
import plotly.graph_objects as go 
import matplotlib.image as mpimg 
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import confusion_matrix, cohen_kappa_score 
import statsmodels.formula.api as smf
from PIL import Image
# Load libraries
sys.path.append('../') 
sys.path.insert(0,'/Users/mlin2/Desktop/RabLab/eoad_sustain_ml/')
import os 
import pySuStaIn  
from plotly.offline import init_notebook_mode  
import plotly.io as pio
from plotly.graph_objs import * 
init_notebook_mode(connected=True)
pio.renderers.default = 'notebook_connected'
# import the python packages needed to generate simulated data for the tutorial
 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_1samp, f_oneway, kurtosis, skew , ttest_ind, norm, t 

# column order 
lobes = ['L_MTL', 'R_MTL', 'L_temporal', 'R_temporal', 'L_frontal', 'R_frontal', 
         'L_occipital', 'R_occipital','L_parietal', 'R_parietal' ]
data_path = "/Users/mlin2/Desktop/RabLab/eoad_sustain_ml/data/"  


In [2]:
# eligibility check
cn_exclude = ["LDS0220099", "LDS0110254", "LDS0670457", "LDS0370222"]
eoad_exclude = ["LDS0100607", "LDS0160473", "LDS0220084", "LDS0220159", "LDS0360398", "LDS0730183", "LDS0980440", "LDS0990439", "LDS1770209", "LDS1770310", "LDS1770403", "LDS3600132", "LDS3600550", "LDS3600671"]
ftp_data = pd.read_csv(data_path+'baseline_FTP_6mm.csv')
# read z-scored data (2GMM w/o EOnonAD standardized)
zdata_full = pd.read_csv(data_path+'full_gmm2z_exclude.csv')
zdata = zdata_full[zdata_full.visit==1]
# to numpy
data = zdata[lobes].values
zdata_ctrl = zdata[zdata.dx == 'CN'][lobes].values
zdata_eoad = zdata[zdata.dx == 'EOAD'][lobes].values
zmax = np.ceil(zdata[lobes].apply(max,axis=0)).values  
thre = pd.read_csv(data_path+'2gmm_thre_exclude.csv') 
#thre = thre[['roi','intersection','c2mean-sd','c2mean+sd']]
thre = thre[['roi','intersection','c2mean']]#thre[['roi','c2mean-sd','c2mean','c2mean+sd']] 
#print('fixed thresholds: 2, 5, 10')
print('ROI-specific thresholds: intersection, c2mean')
pd.concat([thre,pd.DataFrame({'max':zmax})],axis=1)


ROI-specific thresholds: intersection, c2mean


,roi,intersection,c2mean,max
0,L_MTL,1.133637,5.128765,14.0
1,R_MTL,1.254431,4.920137,13.0
2,L_temporal,1.858691,15.198490,38.0
3,R_temporal,1.970307,14.680046,39.0
4,L_frontal,1.487233,10.733511,33.0
5,R_frontal,1.801431,9.984381,34.0
6,L_occipital,2.207846,15.404934,52.0
7,R_occipital,2.044334,14.923082,55.0
8,L_parietal,2.122259,16.583604,43.0
9,R_parietal,2.116481,18.069873,47.0


In [3]:
# Input the settings for z-score SuStaIn 
N = 10         # number of biomarkers
M = 365 # number of observations ( e.g. subjects )
M_control = 0   # number of these that are control subjects
N_startpoints = 25 #25
#N_S_gt = 1 #number of ground truth subtypes
N_S_max = 6 #7
N_iterations_MCMC = int(1e6) #1e4 / 1e6

SuStaInLabels = lobes
Z_vals = thre[['intersection','c2mean']].values#,'c2mean+sd']].values #np.array([[2,5,10]]*N) ## thre[['intersection','c2mean-sd','c2mean+sd']].values  # Z-scores for each biomarker
Z_max = zmax  #z95 #np.array([10]*N)           # maximum z-score 
output_folder = os.path.join('/Users/mlin2/Desktop/RabLab/eoad_sustain_ml/inter_c2mean')#'inter_c2mean_sd_e6'
dataset_name = '2GMMz' 
sustain_input = pySuStaIn.ZscoreSustain(zdata_eoad,
                                             Z_vals,
                                             Z_max,
                                             SuStaInLabels,
                                             N_startpoints,
                                             N_S_max, 
                                             N_iterations_MCMC, 
                                             output_folder, 
                                             dataset_name, 
                                             True)
print(f'reading output from: {output_folder}')

reading output from: /Users/mlin2/Desktop/RabLab/eoad_sustain_ml/inter_c2mean


In [4]:
df = pd.DataFrame(zdata_eoad,columns=lobes)
# The SuStaIn output has everything we need. We'll use it to populate our dataframe.

s = 2
pickle_filename_s = output_folder + '/pickle_files/' + dataset_name + '_subtype' + str(s) + '.pickle'
pk = pd.read_pickle(pickle_filename_s)

for variable in ['ml_subtype', # the assigned subtype
                 'prob_ml_subtype', # the probability of the assigned subtype
                 'ml_stage', # the assigned stage 
                 'prob_ml_stage',]: # the probability of the assigned stage
    
    # add SuStaIn output to dataframe
    df.loc[:,variable] = pk[variable] 

# let's also add the probability for each subject of being each subtype
for i in range(s):
    df.loc[:,'prob_S%s'%i] = pk['prob_subtype'][:,i]

# IMPORTANT!!! The last thing we need to do is to set all "Stage 0" subtypes to their own subtype
# We'll set current subtype (0 and 1) to 1 and 0, and we'll call "Stage 0" individuals subtype 0.

# make current subtypes (0 and 1) 1 and 2 instead
df.loc[:,'ml_subtype'] = df.ml_subtype.values + 1

# convert "Stage 0" subjects to subtype 0
#df.loc[df.ml_stage==0,'ml_subtype'] = 0

# change prob_sx to handle diff number of subtypes
df.reset_index() 
df['prob_S2'] = 1 - df['prob_S0']- df['prob_S1']  #- df['prob_S2'] #- df['prob_S3'] - df['prob_S4'] 
df['subj'] = zdata[zdata.dx=='EOAD'].subj.values

# adjustment to reassign subtype based on prevalence
# Step 1: Count current subtype prevalence
prevalence = df['ml_subtype'].value_counts().sort_values(ascending=False)

# Step 2: Create a mapping for reassigning subtypes in order of prevalence
subtype_mapping = {old: new for new, old in enumerate(prevalence.index, start=1)}

# Step 3: Update ml_subtype using the mapping
df['ml_subtype'] = df['ml_subtype'].map(subtype_mapping)

# Step 4: Update probability columns to reflect the new subtype order
prob_columns = ['prob_S0', 'prob_S1', 'prob_S2']#,'prob_S3']
prob_mapping = {0: 'prob_S0', 1: 'prob_S1', 2: 'prob_S2'}#,3: 'prob_S3'}
# Reassign probabilities based on new subtype mapping
for new_subtype, old_subtype in subtype_mapping.items():
    df[f'prob_s{int(new_subtype) - 1}'] = df[prob_mapping[old_subtype - 1]]

# Drop old probability columns  
for col in prob_columns:
    if col not in [f'prob_s{new - 1}' for new in subtype_mapping.values()]:
        df.drop(columns=col, inplace=True)
#print(df.columns)
print('baseline subtype count:')
df.ml_subtype.value_counts()


baseline subtype count:


ml_subtype
1    148
2    111
3    106
Name: count, dtype: int64

In [7]:
# subtype and stage all scans
new_df = zdata_full[zdata_full.dx == 'EOAD']
typedf = new_df[['subj','visit','ftp_date']]
typedf1 = typedf[typedf.visit==1]
typedf2 = typedf[typedf.visit!=1] 
for i in range(1, N_S_max): #i is the number of split, so starting from 1 split = 2 subtypes
    ##Subtype assignment
    pickle_filename_s = output_folder + '/pickle_files/' + dataset_name + '_subtype' + str(i) + '.pickle'
    pk = pd.read_pickle(pickle_filename_s) 
    subtypes = pk['ml_subtype']
    stages = pk['ml_stage']
    styp_prob = pk['prob_ml_subtype']
    #subtypes = [subtypes[i] if  styp_prob[i] >= 0.5 else 98 for i in range(len(subtypes))]
    typedf1[str(i+1)] = subtypes
    # Add 1 to the subtype (assuming it's indexed from 0 in original)
    typedf1[str(i+1)] = typedf1[str(i+1)].astype(int) + 1
    # Assign the ml_stage
    stage = str(i+1) + 's'
    typedf1[stage] = stages
    typedf1[stage] = typedf1[stage].astype(int) 
    subtype_prob_name = f'{str(i+1)}_subtype_prob'
    typedf1[subtype_prob_name] = pk['prob_ml_subtype']
    stage_prob_name = f'{str(i+1)}_stage_prob'
    typedf1[stage_prob_name] = pk['prob_ml_stage']

    
    pickle_filename_s = output_folder + '/pickle_files/' + dataset_name + '_subtype' + str(i) + '.pickle'
    pk = pd.read_pickle(pickle_filename_s)
    new_dat = new_df[new_df.visit!=1][lobes].values
    s_seq = pk['samples_sequence']
    s_f = pk['samples_f']
    n_s = 700 - 365
    ml_subtype, prob_ml_subtype, ml_stage, prob_ml_stage, prob_subtype, prob_stage, prob_subtype_stage = \
        pySuStaIn.ZscoreSustain.subtype_and_stage_individuals_newData(sustain_input, data_new = new_dat, 
    samples_sequence = s_seq, samples_f = s_f , N_samples = n_s)
    # if the probability is smaller than 0.5, poorly fit
    subtypes = ml_subtype#[ml_subtype[i] if prob_ml_subtype[i] >= 0.5 else 98 for i in range(len(ml_subtype))]
    typedf2[str(i+1)] = subtypes
    # Add 1 to the subtype (assuming it's indexed from 0 in original)
    typedf2[str(i+1)] = typedf2[str(i+1)].astype(int) + 1
    # Assign the ml_stage
    stage = str(i+1) + 's'
    typedf2[stage] = ml_stage
    typedf2[stage] = typedf2[stage].astype(int) 
    subtype_prob_name = f'{str(i+1)}_subtype_prob'
    typedf2[subtype_prob_name] = prob_ml_subtype
    stage_prob_name = f'{str(i+1)}_stage_prob'
    typedf2[stage_prob_name] = prob_ml_stage
typedf = pd.concat([typedf1,typedf2],axis=0,ignore_index=True)
typedf['fname'] = typedf.apply(lambda row:f"wr{row['subj']}_FTP_{row['ftp_date']}_suvr-infcblgm.nii",axis=1)

/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/618022937.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/618022937.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/618022937.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [8]:
ss = typedf.copy()
ss = ss[['subj', 'visit', 'ftp_date', '3', '3s', '3_subtype_prob', '3_stage_prob', 'fname']]
ss.columns = ['subj', 'visit', 'ftp_date', 'subtype', 'stage', 'subtype_prob', 'stage_prob', 'fname']
ss['subtype'] = ss['subtype'].astype(int)
ss['stage'] = ss['stage'].astype(int)
#data = pd.read_csv(data_path + 'full_combined.csv')  # Convert ftp_date to datetime in both DataFrames 
data = pd.read_csv(data_path+'tableone_dat.csv' )
data['ftp_date'] = pd.to_datetime(data['ftp_date'], errors='coerce')
data.drop(columns=["visit",'subtype','stage','subtype_prob','stage_prob','fname'],inplace=True)
ss['ftp_date'] = pd.to_datetime(ss['ftp_date'], errors='coerce')
combined = pd.merge(data, ss, on=['subj', 'ftp_date'], how='right')
combined = combined.sort_values(by=['subj', 'ftp_date']).reset_index(drop=True)
combined['visit'] = combined.groupby('subj').cumcount() + 1
#subtype_mapping = {old: new for new, old in enumerate(prevalence.index, start=1)}
#combined['subtype'] = combined['subtype'].map(subtype_mapping) 
combined_bl = combined[combined.visit==1]
print('baseline prevalence')
print(combined_bl.subtype.value_counts())
print(f'total scans: {len(combined)}')
print('variables:')
print(combined_bl.columns)


baseline prevalence
subtype
1    148
2    111
3    106
Name: count, dtype: int64
total scans: 699
variables:
Index(['subj', 'fbb_date', 'amy_file', 'ftp_file', 'ftp_date', 'event',
       'SUVR Frontal', 'SUVR MesialTemporal', 'SUVR MetaTemporal',
       'SUVR TemporoParietal',
       ...
       'ctx_desikan_MRIBASED_SUVR', 'Benson_Forgetting', 'RAVLT_Forgetting',
       'Craft_Forgetting', 'visit', 'subtype', 'stage', 'subtype_prob',
       'stage_prob', 'fname'],
      dtype='object', length=105)


/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/2427163334.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



# excluded

cont = [ 'Centiloids MRI-Based Composite',
       'Yrs. of Education','Age', 'CDR-SB',
       'MMSE','stage',  'SUVR Universal','ctx_desikan_MRIBASED_SUVR','meta_temporal_MRIBASED_SUVR', 
'MOCATOTS']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Cognitive Behavior'] 
cols_to_keep = cont+cat
thre5 = combined_bl[combined_bl.subtype_prob<0.5][cols_to_keep]
thre5['threshold'] = '0.5'
thre8 = combined_bl[combined_bl.subtype_prob<0.8][cols_to_keep]
thre8['threshold'] = '0.8'
print('thre5 shape:', thre5.shape)
print('thre8 shape:', thre8.shape)

tdata = pd.concat([thre5,thre8], axis=0, ignore_index=True)
print(tdata.shape) 

tdata['Clinical Phenotype'] = tdata['Cognitive Behavior']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Clinical Phenotype'] 
t1 = TableOne(data=tdata, columns=cont+cat, categorical=cat, groupby='threshold', pval=True,row_percent=False)


In [9]:
cont = [ 'Centiloids MRI-Based Composite',
       'Yrs. of Education','Age', 'CDR-SB',
       'MMSE','stage',  'SUVR Universal','ctx_desikan_MRIBASED_SUVR','meta_temporal_MRIBASED_SUVR', 
'MOCATOTS']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Cognitive Behavior'] 
cols_to_keep = cont+cat
thre5 = combined_bl[cols_to_keep]
thre5['exclude'] = combined_bl.subtype_prob<0.5

thre8 = combined_bl[cols_to_keep]
thre8['exclude']= combined_bl.subtype_prob<0.8 

/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/1321635764.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/1321635764.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
from tableone import TableOne

In [13]:

thre5['Clinical Phenotype'] = thre5['Cognitive Behavior']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Clinical Phenotype'] 
t1 = TableOne(data=thre5, columns=cont+cat, categorical=cat, groupby='exclude', pval=True,row_percent=False,nonnormal=cont,htest_name=True)


/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/946473909.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/envs/sustain/lib/python3.12/site-packages/tableone/tableone.py:322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
t1 

Grouped by exclude                                                                                                             
                                                                                                        Missing             Overall               False              True P-Value                                       Test
n                                                                                                                               365                 359                 6                                                   
Centiloids MRI-Based Composite, median [Q1,Q3]                                                                0  103.3 [83.2,121.1]  103.8 [83.9,121.3]  64.7 [59.9,70.5]   0.001                             Kruskal-Wallis
Yrs. of Education, median [Q1,Q3]                                                                             0    16.0 [14.0,18.0]    16.0 [14.0,18.0]  15.0 [13.2,16.0]   0.431                             Kruskal-Wallis
Age, median [Q1,Q3]                                                                                           0    59.4 [56.5,62.5]    59.3 [56.5,62.4]  64.5 [64.2,64.8]   0.002                             Kruskal-Wallis
CDR-SB, median [Q1,Q3]                                                                                        3       4.0 [2.5,5.0]       4.0 [2.5,5.0]     2.0 [1.5,2.9]   0.041                             Kruskal-Wallis
MMSE, median [Q1,Q3]                                                                                          9    22.0 [18.0,25.0]    22.0 [18.0,25.0]  27.5 [26.2,28.0]   0.005                             Kruskal-Wallis
stage, median [Q1,Q3]                                                                                         0    13.0 [10.0,14.0]    13.0 [10.0,15.0]     0.0 [0.0,0.0]  <0.001                             Kruskal-Wallis
SUVR Universal, median [Q1,Q3]                                                                                0       2.6 [2.1,2.9]       2.6 [2.2,2.9]     1.2 [1.1,1.2]  <0.001                             Kruskal-Wallis
ctx_desikan_MRIBASED_SUVR, median [Q1,Q3]                                                                     0       1.9 [1.6,2.2]       1.9 [1.6,2.2]     1.1 [1.0,1.1]  <0.001                             Kruskal-Wallis
meta_temporal_MRIBASED_SUVR, median [Q1,Q3]                                                                   0       2.2 [1.9,2.5]       2.2 [1.9,2.5]     1.2 [1.1,1.2]  <0.001                             Kruskal-Wallis
MOCATOTS, median [Q1,Q3]                                                                                     28    16.0 [11.0,20.0]    16.0 [11.0,20.0]  20.0 [19.2,20.8]   0.082                             Kruskal-Wallis
Gender, n (%)                                  Female                                                                    201 (55.1)          198 (55.2)          3 (50.0)   1.000                             Fisher's exact
                                               Male                                                                      164 (44.9)          161 (44.8)          3 (50.0)                                                   
Diagnosis, n (%)                               Dementia                                                                  271 (74.2)          267 (74.4)          4 (66.7)   0.900  Chi-squared (warning: expected count < 5)
                                               MCI                                                                        93 (25.5)           91 (25.3)          2 (33.3)                                                   
                                               None                                                                         1 (0.3)             1 (0.3)                                                                     
ApoE4 Carrier, n (%)                           Carrier                                                             

In [18]:

thre8['Clinical Phenotype'] = thre8['Cognitive Behavior']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Clinical Phenotype'] 
t1 = TableOne(data=thre8, columns=cont+cat, categorical=cat, groupby='exclude', pval=True,row_percent=False,nonnormal=cont,htest_name=True)


/var/folders/4_/cjf386ls6qbdt4chbrjvnhfm0000gn/T/ipykernel_24229/1845024719.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/envs/sustain/lib/python3.12/site-packages/tableone/tableone.py:322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
t1

Grouped by exclude                                                                                                              
                                                                                                        Missing             Overall               False               True P-Value                                       Test
n                                                                                                                               365                 336                 29                                                   
Centiloids MRI-Based Composite, median [Q1,Q3]                                                                0  103.3 [83.2,121.1]  104.0 [85.9,122.3]  71.4 [60.5,101.9]  <0.001                             Kruskal-Wallis
Yrs. of Education, median [Q1,Q3]                                                                             0    16.0 [14.0,18.0]    16.0 [14.0,18.0]   16.0 [14.0,16.0]   0.508                             Kruskal-Wallis
Age, median [Q1,Q3]                                                                                           0    59.4 [56.5,62.5]    59.0 [56.3,62.3]   62.4 [59.9,64.2]  <0.001                             Kruskal-Wallis
CDR-SB, median [Q1,Q3]                                                                                        3       4.0 [2.5,5.0]       4.0 [2.5,5.0]      2.5 [1.5,3.0]  <0.001                             Kruskal-Wallis
MMSE, median [Q1,Q3]                                                                                          9    22.0 [18.0,25.0]    22.0 [18.0,25.0]   26.0 [23.0,29.0]  <0.001                             Kruskal-Wallis
stage, median [Q1,Q3]                                                                                         0    13.0 [10.0,14.0]    13.0 [11.0,15.0]     2.0 [0.0,10.0]  <0.001                             Kruskal-Wallis
SUVR Universal, median [Q1,Q3]                                                                                0       2.6 [2.1,2.9]       2.6 [2.2,3.0]      1.3 [1.2,2.0]  <0.001                             Kruskal-Wallis
ctx_desikan_MRIBASED_SUVR, median [Q1,Q3]                                                                     0       1.9 [1.6,2.2]       1.9 [1.7,2.2]      1.2 [1.1,1.6]  <0.001                             Kruskal-Wallis
meta_temporal_MRIBASED_SUVR, median [Q1,Q3]                                                                   0       2.2 [1.9,2.5]       2.3 [1.9,2.6]      1.4 [1.2,1.7]  <0.001                             Kruskal-Wallis
MOCATOTS, median [Q1,Q3]                                                                                     28    16.0 [11.0,20.0]    16.0 [11.0,20.0]   21.0 [17.0,22.0]  <0.001                             Kruskal-Wallis
Gender, n (%)                                  Female                                                                    201 (55.1)          187 (55.7)          14 (48.3)   0.567                                Chi-squared
                                               Male                                                                      164 (44.9)          149 (44.3)          15 (51.7)                                                   
Diagnosis, n (%)                               Dementia                                                                  271 (74.2)          256 (76.2)          15 (51.7)   0.013  Chi-squared (warning: expected count < 5)
                                               MCI                                                                        93 (25.5)           79 (23.5)          14 (48.3)                                                   
                                               None                                                                         1 (0.3)             1 (0.3)                                                                      
ApoE4 Carrier, n (%)                           Carrier                                           

# changed 

In [115]:
# longitudinal data: exclude those that were poorly classified at baseline 
# pre-processing
# 1. exclude sta  ge 0 or poorly fit p < 0.5 at V1 
print(len(combined))
print(len(combined.subj.unique()))
combined_exclude_long = combined.sort_values(by=['subj', 'ftp_date']).copy()
# Filter out subjects with stage 0 or subtype probability < 0.5 at the first visit
first_visit = combined_exclude_long.groupby('subj').first().reset_index()
exclude_mask = (first_visit['subtype_prob'] < 0.5) | (first_visit['stage'] == 0.) 
combined_exclude_long = combined_exclude_long[~combined_exclude_long['subj'].isin(first_visit.loc[exclude_mask, 'subj'])]
print(f'total number of scans after excluding scans from subjects that are poorly & stage 0 assigned at baseline: {len(combined_exclude_long)}')
print(f'total number of subjects after excluding scans from subjects that are poorly assigned & stage 0 at baseline: {len(combined_exclude_long.subj.unique())}')

699
365
total number of scans after excluding scans from subjects that are poorly & stage 0 assigned at baseline: 682
total number of subjects after excluding scans from subjects that are poorly assigned & stage 0 at baseline: 356


In [116]:
# anti amyloid
treatment = pd.read_csv(data_path + 'treatment_exclude.csv')
treatment['startdate'] = treatment['startdate'].str.replace('xx', '01', regex=False)
treatment['startdate'] = pd.to_datetime(treatment['startdate'], format='%Y-%m-%d', errors='coerce')

combined_exclude_long['ftp_date'] = pd.to_datetime(combined_exclude_long['ftp_date'], format='%Y-%m-%d', errors='coerce')
treatment_filtered = treatment[~treatment.txname.isna()][['subject_code', 'startdate']]
treatment_filtered['subject_code'] = treatment_filtered['subject_code'].str.upper()
merged_df = combined_exclude_long.merge(
    treatment_filtered, 
    left_on='subj', 
    right_on='subject_code', 
    how='inner'
)
scans_to_exclude = merged_df[merged_df['ftp_date'] > merged_df['startdate']] 
scans_to_exclude[['subj', 'ftp_date', 'startdate']]
# Exclude scans from combined_exclude_long that match subj and ftp_date in scans_to_exclude
print(len(combined_exclude_long.subj.unique()))
print(len(scans_to_exclude.subj.unique()))

356
15


In [117]:
combined_exclude_long = combined_exclude_long.merge(
    scans_to_exclude[['subj', 'ftp_date']],
    on=['subj', 'ftp_date'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge']) 


In [118]:

# Subject IDs before exclusion
before_subjects = set(merged_df['subj'].unique())
# Subject IDs after exclusion
after_subjects = set(combined_exclude_long['subj'].unique())
# Which subjects were dropped
dropped_subjects = before_subjects - after_subjects
print("Subjects dropped:", dropped_subjects)
print("Number dropped:", len(dropped_subjects))

Subjects dropped: {'LDS0360520', 'LDS9410450'}
Number dropped: 2


In [119]:
# exclude poorly fitted longtudinal scans
combined_exclude_long.poorly = combined_exclude_long.subtype_prob < 0.5
combined_exclude_long = combined_exclude_long[~combined_exclude_long.poorly]
print(len(combined_exclude_long))
print(len(combined_exclude_long.subj.unique()))

660
354


In [120]:
# exclude wrong date subjects
wrong_date = ['LDS9410287', 'LDS9410396']
combined_exclude_long = combined_exclude_long[~combined_exclude_long.subj.isin(wrong_date)]
len(combined_exclude_long.subj.unique())

352

In [121]:
d_changed = combined_exclude_long.copy()
# Ensure 'ftp_date' is datetime and 'stage' is numeric
d_changed['ftp_date'] = pd.to_datetime(d_changed['ftp_date'], errors='coerce')
d_changed['stage'] = pd.to_numeric(d_changed['stage'], errors='coerce') 
# Calculate first and last stage for each subject
first_visit = d_changed.groupby('subj').first().reset_index()
last_visit = d_changed.groupby('subj').last().reset_index()
# Merge first and last visit information
progression_data = pd.merge(
    first_visit[['subj', 'subtype', 'stage', 'ftp_date', 'subtype_prob', 'stage_prob']],
    last_visit[['subj', 'subtype', 'stage', 'ftp_date', 'subtype_prob', 'stage_prob']],
    on='subj', suffixes=('_first', '_last')
)

# Calculate yearly stage change
progression_data['years_diff'] = (progression_data['ftp_date_last'] - progression_data['ftp_date_first']).dt.days / 365
progression_data['rate_of_progression'] = (progression_data['stage_last'] - progression_data['stage_first']) / progression_data['years_diff']

# Identify subjects who changed subtype
progression_data['changed_subtype'] = progression_data['subtype_first'] != progression_data['subtype_last']

# Add "Changed Subtype" as a category for plotting
progression_data['subtype_plot'] = progression_data['subtype_first'].astype(str)
progression_data.loc[progression_data['changed_subtype'], 'subtype_plot'] = 'Changed Subtype'

# Filter the baseline visit (most recent = visit 1)
baseline_data = d_changed[d_changed['visit'] == 1]

# Merge progression data with baseline data
merged_data = pd.merge(progression_data, baseline_data, on='subj',how='left')
# only those with longitudinal
longitudinal_subjects = d_changed['subj'].value_counts()
longitudinal_subjects = longitudinal_subjects[longitudinal_subjects > 1].index
merged_data = merged_data[merged_data['subj'].isin(longitudinal_subjects)]
variables_to_compare = [  'stage_first',  
       'subtype_prob_first', 'stage_prob_first',  
       'rate_of_progression',  'SUVR_TemporoParietal', 'SUVR_Universal', 
       'SUVR_PET_Only_Composite_baseline', 'SUVR_MRI_Based_Composite_baseline', 
       'Centiloids_MRI_Based_Composite_baseline', 'MMSE_baseline',  
       'CDR_SB_baseline',
       'Age_baseline', 'Yrs_of_Education_baseline',
       'ApoE4_Genotype',  'Gender', 'Diagnosis_baseline']
merged_data.shape

(201, 119)

In [124]:
cont = ['stage_first',  'Centiloids MRI-Based Composite',
       'Yrs. of Education','Age', 'CDR-SB',
       'MMSE','stage',  'SUVR Universal','ctx_desikan_MRIBASED_SUVR','meta_temporal_MRIBASED_SUVR', 
'MOCATOTS']
cat = ['Gender', 'Diagnosis', 'ApoE4 Carrier','Cognitive Behavior'] 
TableOne(data=merged_data, columns=cont+cat, categorical=cat, groupby='changed_subtype', pval=True,row_percent=False)

Grouped by changed_subtype                                                  
                                                                                                           Missing       Overall         False          True P-Value
n                                                                                                                            201           172            29        
stage_first, mean (SD)                                                                                           0    12.3 (3.5)    12.5 (3.5)    10.9 (3.3)   0.024
Centiloids MRI-Based Composite, mean (SD)                                                                        0  105.3 (27.9)  105.8 (27.5)  102.4 (30.6)   0.578
Yrs. of Education, mean (SD)                                                                                     0    15.5 (2.4)    15.6 (2.4)    14.8 (2.6)   0.129
Age, mean (SD)                                                                                                   0    58.9 (4.1)    58.9 (4.2)    59.5 (3.5)   0.398
CDR-SB, mean (SD)                                                                                                0     3.7 (1.7)     3.7 (1.6)     3.6 (2.0)   0.817
MMSE, mean (SD)                                                                                                  2    21.6 (4.9)    21.7 (4.8)    21.3 (5.2)   0.766
stage, mean (SD)                                                                                                 0    12.3 (3.5)    12.5 (3.5)    10.9 (3.3)   0.024
SUVR Universal, mean (SD)                                                                                        0     2.5 (0.6)     2.6 (0.6)     2.4 (0.5)   0.066
ctx_desikan_MRIBASED_SUVR, mean (SD)                                                                             0     1.9 (0.4)     2.0 (0.4)     1.8 (0.4)   0.137
meta_temporal_MRIBASED_SUVR, mean (SD)                                                                           0     2.2 (0.5)     2.2 (0.5)     2.1 (0.4)   0.058
MOCATOTS, mean (SD)                                                                                             11    15.6 (5.8)    15.8 (5.7)    14.4 (6.4)   0.267
Gender, n (%)                             Female                                                                      110 (54.7)     89 (51.7)     21 (72.4)   0.062
                                          Male                                                                         91 (45.3)     83 (48.3)      8 (27.6)        
Diagnosis, n (%)                          Dementia                                                                    152 (75.6)    129 (75.0)     23 (79.3)   0.790
                                          MCI                                                                          49 (24.4)     43 (25.0)      6 (20.7)        
ApoE4 Carrier, n (%)                      Carrier                                                                     105 (52.2)     93 (54.1)     12 (41.4)   0.346
                                          Non carrier                                                                  94 (46.8)     77 (44.8)     17 (58.6)        
                                          None                                                                           2 (1.0)       2 (1.2)                      
Cognitive Behavior, n (%)                 Amnestic-predominant CI (single/multi-domain)                               157 (78.1)    136 (79.1)     21 (72.4)   0.220
                                          Non-amnestic CI (single/multi-domain)                                         14 (7.0)      12 (7.0)       2 (6.9)        
                                          PCA                                                                           14 (7.0)      13 (7.6)       1 (3.4)        
                                          PPA                                                                           16 (8.0)